<a href="https://colab.research.google.com/github/schandar/IO-Tables/blob/master/IO_Tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input-Output Analysis

In [262]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [263]:
!pip install chart_studio

In [264]:
import chart_studio
import chart_studio.plotly as py
username = 'schandar'
api_key = 'C97wfbyw53eqkApqPWSP'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [265]:
z = pd.read_csv("https://raw.githubusercontent.com/schandar/IO-Tables/master/Z_matrix.csv", header=[0,1], index_col=[0,1])
x = pd.read_csv("https://raw.githubusercontent.com/schandar/IO-Tables/master/X_vector.csv", header = None, index_col = [0, 1])
y = pd.read_csv("https://raw.githubusercontent.com/schandar/IO-Tables/master/Y_vector.csv", header=[0], index_col=[0,1])

In [266]:
z.head()

AUS                                  ... ROW                  
               A01  A02  A03     B C10-C12 C13-C15  ... O84 P85   Q R_S  T  U
AUS A01      12924  112  228   539   25139     737  ...   2  13  13   6  0  0
    A02         83  201    0    17       3       0  ...   0   0   0   0  0  0
    A03         19    0   19     4     316      23  ...   0   0   0   0  0  0
    B          116    1    6  4492     332       6  ...  23  28  54  18  0  0
    C10-C12   1591    1   43   203   10852     262  ...  17  30  24  30  0  0

[5 rows x 2464 columns]

In [267]:
x.head()

2
0   1              
AUS A01       70292
    A02        2585
    A03        3175
    B        171985
    C10-C12   83504

In [268]:
y.head()

AUS       AUT  ...          USA          ROW
AUS A01      13021.267270  0.576625  ...    57.633086   757.877499
    A02        257.476610  0.001655  ...    15.306080     0.424362
    A03       1442.014903  0.001036  ...     0.325774   404.404713
    B        10339.178940  0.141408  ...    12.963735    34.917758
    C10-C12  30139.503480  3.317939  ...  2199.364452  4858.626245

[5 rows x 44 columns]

In [269]:
x[2]

0    1      
AUS  A01         70292
     A02          2585
     A03          3175
     B          171985
     C10-C12     83504
                 ...  
ROW  P85        625260
     Q          511800
     R_S        457353
     T           19919
     U              32
Name: 2, Length: 2464, dtype: int64

Generate A matrix, aij = zij/xj. Aij = amount of inputs required from industry i to industry j, in order to produce one unit of output from industry j.

In [270]:
x_transpose = x.transpose()
a = z.div(x_transpose.iloc[0])
a.fillna(0, inplace=True)
a.head()

AUS                      ...       ROW          
                  A01       A02       A03  ...       R_S    T    U
AUS A01      0.183862  0.043327  0.071811  ...  0.000013  0.0  0.0
    A02      0.001181  0.077756  0.000000  ...  0.000000  0.0  0.0
    A03      0.000270  0.000000  0.005984  ...  0.000000  0.0  0.0
    B        0.001650  0.000387  0.001890  ...  0.000039  0.0  0.0
    C10-C12  0.022634  0.000387  0.013543  ...  0.000066  0.0  0.0

[5 rows x 2464 columns]

Create Leontief Inverse matrix, L = (I-A)^(-1)

In [271]:
i = a.copy()
i[:] = np.identity(2464)
i_a = i.subtract(a)
l = pd.DataFrame(np.linalg.pinv(i_a.values), i_a.columns, i_a.index)
l.head()

AUS                      ...       ROW                        
                  A01       A02       A03  ...       R_S             T         U
AUS A01      1.240736  0.059541  0.097336  ...  0.000265  3.441071e-17  0.000127
    A02      0.002167  1.084474  0.000370  ...  0.000009 -1.515173e-17  0.000006
    A03      0.000635  0.000080  1.006229  ...  0.000005  1.833132e-16  0.000002
    B        0.016844  0.050831  0.025665  ...  0.001484 -2.788432e-17  0.001723
    C10-C12  0.036037  0.003205  0.020391  ...  0.000276  1.544988e-17  0.000103

[5 rows x 2464 columns]

Baseline X and Y:


In [272]:
y_baseline = pd.DataFrame(y.sum(axis = 1))
y_baseline.index.set_names(["country", "code"], inplace=True)
y_baseline.head()
x_baseline = pd.DataFrame(l.dot(y.sum(axis = 1)))
x_baseline.columns = ['output']
x_baseline.index.set_names(["country", "code"], inplace=True)

Industry Codes:

In [273]:
codes = pd.read_csv("https://raw.githubusercontent.com/schandar/IO-Tables/master/industry_codes.csv", header = 0, index_col = 0)
code_dict = codes.to_dict()['industry']
code_dict

fig = go.Figure(data=[go.Table(columnwidth = [80,400],
    header=dict(values=['Industry Code', 'Description'],
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[list(code_dict.keys()), list(code_dict.values())],
               fill_color='white',
               align='left'))
])

fig.show()

In [274]:
py.plot(fig, filename = 'industry', auto_open=True)

'https://plotly.com/~schandar/32/'

Country Code + Above or Below World Average in Corona Cases per 1M population 
** data sourced from Worldometer **

In [275]:
ab_df = pd.read_csv("https://raw.githubusercontent.com/schandar/IO-Tables/master/CoronaCasesCountry.csv", index_col = 0, header = None, names = ['country', 'full', 'above'])
ab_df.head()

,full,above
country,,
AUS,Australia,0
AUT,Austria,1
BEL,Belgium,1
BGR,Bulgaria,0
BRA,Brazil,1


In [276]:
y_baseline = y_baseline.join(ab_df, how = 'inner')

In [277]:
y_baseline.columns = ['fd', 'name', 'above']
y_baseline.head()

fd       name  above
country code                                   
AUS     A01      14574.085833  Australia      0
        A02        275.326425  Australia      0
        A03       1853.489861  Australia      0
        B        10812.901660  Australia      0
        C10-C12  45622.824095  Australia      0

In [278]:
def adjust_df(df, perc_above, perc_below, c_code):
  df.fd = np.where((df.above == 1) & (df.index.get_level_values('code') == c_code), df.fd*(1-perc_above), df.fd)
  df.fd = np.where((df.above == 0) & (df.index.get_level_values('code') == c_code), df.fd*(1-perc_below), df.fd)

In [279]:
x_baseline.head()

output
country code                  
AUS     A01       70160.775209
        A02        2556.695826
        A03        3161.162180
        B        171632.154578
        C10-C12   83403.534165

In [280]:
td = x_baseline.groupby(["country"]).sum()

In [281]:
td.columns = ['total']
td.head()

,total
country,
AUS,2.718785e+06
AUT,8.008932e+05
BEL,1.102013e+06
BGR,1.185085e+05
BRA,4.098565e+06


In [282]:
td[td.index == "MEX"]

,total
country,
MEX,2.127870e+06


In [283]:
def create_diff_df(y_scenario):
  x_scenario = pd.DataFrame(l.dot(y_scenario))
  x_scenario.columns = ['output']
  diff_scenario = x_scenario.subtract(x_baseline)
  diff_scenario = diff_scenario.join(td, how = 'inner')
  diff_scenario['output'] = diff_scenario.output/diff_scenario.total
  diff_scenario.index.set_names(["country", "code"], inplace=True)
  diff_scenario = diff_scenario.join(ab_df, how = 'inner')
  return diff_scenario

In [284]:
countries = ab_df.full.tolist()
industries = list(code_dict.keys())

In [299]:
countries[16] = "United Kingdom"

In [300]:
def create_plot(diff_scenario):
  outputs = diff_scenario.output.tolist()
  data_ls = []
  for k in code_dict:
    outputs = diff_scenario[diff_scenario.index.get_level_values('code') == k].output.tolist()
    data_ls.append(go.Bar(name = k, x = countries, y = outputs))

  fig = go.Figure(data=data_ls)
  # Change the bar mode
  fig.update_layout(barmode='stack')
  fig.show()
  return fig

In [301]:
x_baseline.index.set_names(["country", "code"], inplace=True)
fig = create_plot(x_baseline)

In [302]:
py.plot(fig, filename = 'baseline', auto_open=True)

'https://plotly.com/~schandar/1/'

In [303]:
code_dict['Q']

'Human health and social work activities'

Scenario 1: Real Estate spending declines by 30%


In [304]:
y_scenario1 = y_baseline.copy()
adjust_df(y_scenario1, .3, .3, "L68")
y_scenario1 = y_scenario1[['fd']]
diff_scenario1 = create_diff_df(y_scenario1)
fig = create_plot(diff_scenario1)
py.plot(fig, filename = 'scenario1', auto_open=True)

'https://plotly.com/~schandar/5/'

Scenario 2: Tourism spending declines 50%

In [305]:
y_scenario2 = y_baseline.copy()
adjust_df(y_scenario2, .5, .5, "I")
y_scenario2 = y_scenario2[['fd']]
diff_scenario2 = create_diff_df(y_scenario2)
fig = create_plot(diff_scenario2)
py.plot(fig, filename = 'scenario2', auto_open=True)

'https://plotly.com/~schandar/24/'

In [306]:
code_dict["C10-C12"]

'Manufacture of food products, beverages and tobacco products'

Scenario 3: Retail spending declines by 12%

In [307]:
y_scenario3 = y_baseline.copy()
adjust_df(y_scenario3, .12, .12, "G47")
y_scenario3 = y_scenario3[['fd']]
diff_scenario3 = create_diff_df(y_scenario3)
fig = create_plot(diff_scenario3)
py.plot(fig, filename = 'scenario3', auto_open=True)

'https://plotly.com/~schandar/26/'

Scenario 4: Tech sector increases by 25%

In [308]:
y_scenario4 = y_baseline.copy()
adjust_df(y_scenario4, -.25, -.25, "J62_J63")
y_scenario4 = y_scenario4[['fd']]
diff_scenario4 = create_diff_df(y_scenario4)
fig = create_plot(diff_scenario4)
py.plot(fig, filename = 'scenario4', auto_open=True)

'https://plotly.com/~schandar/28/'

In [309]:
code_dict['N']

'Administrative and support service activities'

Scenario 5: All scenarios happen at once

In [310]:
y_scenario5 = y_baseline.copy()
adjust_df(y_scenario5, .3, .3, "L68")
adjust_df(y_scenario5, .5, .5, "I")
adjust_df(y_scenario5, .12, .12, "G47")
adjust_df(y_scenario5, -.25, -.25, "J62_J63")
y_scenario5 = y_scenario5[['fd']]
diff_scenario5 = create_diff_df(y_scenario5)
fig = create_plot(diff_scenario5)
py.plot(fig, filename = 'scenario5', auto_open=True)

'https://plotly.com/~schandar/30/'